In [ ]:
single domain antibody, how to parse

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [10]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

#
pdb_dir = './pdb'
data_dir = './data'
outputs_dir = './outputs'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex/study_complex', '/home/yuan/bio/antibody-antigen-complex/study_complex/src', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
pdb_acc = '1IAI'

In [4]:
# get *pdb
pdb_file = f'pdb/{pdb_acc}.pdb'
if not os.path.isfile(pdb_file):
    print(f"download {pdb_acc} to {pdb_file}")
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_acc}.pdb -P ./pdb")
print(f"{pdb_file} is ready")

download 1IAI to pdb/1IAI.pdb
pdb/1IAI.pdb is ready


In [5]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(f'pdb/{pdb_acc}.pdb').read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex


In [11]:
# get chains
from src.parse_abag import ParseAbAg

p = ParseAbAg(f'./pdb/{pdb_acc}.pdb', outputs_dir)

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1IAI
outputs dir:  ./outputs/1I/1IAI
[{'chains': [{'AA residues': 214,
              'chain_id': 'L',
              'seq': 'DIVMTQSHKFMSTSVGDRVSITCKASQDVSTAVAWYQQKPGQSPKLLIYSASYQYTGVPDRFTGSGSRTDFTFTINSVQAEDLAVYYCHQHYSTPFTFGSGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC'},
             {'AA residues': 219,
              'chain_id': 'H',
              'seq': 'QIQLVQSGPELKKPGETVKISCKASGYTFTNYGMNWVKQAPGKGLKWMAWINTYTGEPTYADDFKGRFAFSLETSASTAYLQINNLKNEDTATYFCARDGYYENYYAMDYWGQGTSVTVSSAKTTAPSVYPLAPVCGDTTGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVTSSTTPSQSITCNVAHPASSTKVDKKID'},
             {'AA residues': 215,
              'chain_id': 'M',
              'seq': 'DIQLTQSPAFMAASPGEKVTITCSVSSSISSSNLHWYQQKSETSPKPWIYGTSNLASGVPVRFSGSGSGTSYSLTISSMEAEDAATYYCQQWNSYPYTFGGGTKLEIKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC'},
       

In [12]:
print(p.structure.header['keywords'])
print(p.structure_id)

complex (immunoglobulin igg1-igg2a), complex (immunoglobulin igg1- igg2a) complex
1IAI


In [13]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'idiotypic fab 730.1.4 (igg1) of virus neutralizingantibody ',
  'chain': 'l'},
 '2': {'misc': '',
  'molecule': 'idiotypic fab 730.1.4 (igg1) of virus neutralizingantibody ',
  'chain': 'h'},
 '3': {'misc': '',
  'molecule': 'anti-idiotypic fab 409.5.3 (igg2a)',
  'chain': 'm'},
 '4': {'misc': '',
  'molecule': 'anti-idiotypic fab 409.5.3 (igg2a)',
  'chain': 'i'}}

In [14]:
# filter: is antibody-antigen complex
info = p.filter_antibody_antigen()
print(info)
p.update_chains('chain_type')

None


In [130]:
# split chains
p.chains_to_pdb()

Saved chain ['L'] to ./outputs/1I/1IAI/1IAI_L.pdb
Saved chain ['M'] to ./outputs/1I/1IAI/1IAI_M.pdb
Saved chain ['L', 'M'] to ./outputs/1I/1IAI/1IAI_L_M.pdb
Saved chain ['I'] to ./outputs/1I/1IAI/1IAI_I.pdb
Saved chain ['L', 'I'] to ./outputs/1I/1IAI/1IAI_L_I.pdb
Saved chain ['H'] to ./outputs/1I/1IAI/1IAI_H.pdb
Saved chain ['H', 'M'] to ./outputs/1I/1IAI/1IAI_H_M.pdb
Saved chain ['H', 'I'] to ./outputs/1I/1IAI/1IAI_H_I.pdb


In [117]:
p.bounded

[{'type': 'antibody-receptor',
  'chains': [['L'], ['M'], ['L', 'M']],
  'pdb_files': ['./outputs/1I/1IAI/1IAI_L.pdb',
   './outputs/1I/1IAI/1IAI_M.pdb',
   './outputs/1I/1IAI/1IAI_L_M.pdb']},
 {'type': 'antibody-receptor',
  'chains': [['L'], ['I'], ['L', 'I']],
  'pdb_files': ['./outputs/1I/1IAI/1IAI_L.pdb',
   './outputs/1I/1IAI/1IAI_I.pdb',
   './outputs/1I/1IAI/1IAI_L_I.pdb']},
 {'type': 'antibody-receptor',
  'chains': [['H'], ['M'], ['H', 'M']],
  'pdb_files': ['./outputs/1I/1IAI/1IAI_H.pdb',
   './outputs/1I/1IAI/1IAI_M.pdb',
   './outputs/1I/1IAI/1IAI_H_M.pdb']},
 {'type': 'antibody-receptor',
  'chains': [['H'], ['I'], ['H', 'I']],
  'pdb_files': ['./outputs/1I/1IAI/1IAI_H.pdb',
   './outputs/1I/1IAI/1IAI_I.pdb',
   './outputs/1I/1IAI/1IAI_H_I.pdb']}]

## load SASA

single domain ~ receptor
A ~ X
B ~ Y

In [131]:
# calculate SASA
p.build_freesasa()

    chain_id res_no  res aa      value
104        I    105  TYR  Y  93.473917
105        I    106  TYR  Y  21.585456


'./outputs/1I/1IAI/freesasa.p'

In [132]:
from analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa('./outputs/1I/1IAI/freesasa.p')
p2.print_meta()

number of complex 3
antibody chain L ~ receptor chain M
antibody chain H ~ receptor chain M
antibody chain H ~ receptor chain I


In [121]:
# complex
p2.print_delta_sasa(0)

complex: antibody chain L ~ receptor chain M
------------------------------
antibody chain:
   chain_id res_no  res aa       value
26        L     27  GLN  Q    6.995296
27        L     28  ASP  D   20.225661
29        L     30  SER  S    5.139394
67        L     68  ARG  R   34.840011
90        L     91  HIS  H    1.998656
91        L     92  TYR  Y  102.430874
92        L     93  SER  S   33.641531
93        L     94  THR  T    8.623395
------------------------------
receptor chain:
   chain_id res_no  res aa      value
26        M     27  SER  S  20.825944
27        M     28  SER  S  63.491386
28        M     29  ILE  I   9.544272
29        M     30  SER  S  49.404384
30        M     31  SER  S  33.428858
31        M     32  SER  S   3.865215
52        M     53  SER  S   2.055758
67        M     68  SER  S  21.038618
68        M     69  GLY  G   4.055821
92        M     93  ASN  N   3.997312


In [122]:
# complex
p2.print_delta_sasa(1)

complex: antibody chain H ~ receptor chain M
------------------------------
antibody chain:
    chain_id res_no  res aa      value
54         H     55  THR  T   3.407698
56         H     57  GLU  E  17.575225
58         H     59  THR  T  10.278789
103        H    104  ASN  N  42.704410
104        H    105  TYR  Y  41.769560
105        H    106  TYR  Y   8.783793
------------------------------
receptor chain:
   chain_id res_no  res aa      value
29        M     30  SER  S  10.815523
31        M     32  SER  S   4.435577
32        M     33  ASN  N  14.476610
49        M     50  TYR  Y  16.446062
53        M     54  ASN  N  17.420006
92        M     93  ASN  N  52.775087
93        M     94  SER  S   5.407762
94        M     95  TYR  Y   1.027879
96        M     97  TYR  Y   3.083637


In [123]:
# complex
p2.print_delta_sasa(2)

complex: antibody chain H ~ receptor chain I
------------------------------
antibody chain:
    chain_id res_no  res aa       value
29         H     30  THR  T   10.992609
30         H     31  ASN  N   96.084832
31         H     32  TYR  Y   39.402219
32         H     33  GLY  G    9.671883
49         H     50  TRP  W   69.337664
51         H     52  ASN  N   44.275672
52         H     53  THR  T    2.322667
53         H     54  TYR  Y  111.091064
54         H     55  THR  T    8.111642
58         H     59  THR  T   24.881766
100        H    101  TYR  Y   88.418278
101        H    102  TYR  Y    2.703881
105        H    106  TYR  Y   98.289250
------------------------------
receptor chain:
    chain_id res_no  res aa       value
0          I      1  GLU  E   32.349608
1          I      2  VAL  V    1.351940
25         I     26  GLY  G    2.997984
27         I     28  THR  T   14.218999
28         I     29  PHE  F   50.581552
29         I     30  ASN  N   25.132741
30         I     31  

In [124]:
# retrieve sequences
motifs = p2.retrieve_seq()
motifs

([{'antibody': [{'seq': 'QDVS',
     'sig_res': 3,
     'start': (26, '27'),
     'end': (29, '30')},
    {'seq': 'RTDFTFTINSVQAEDLAVYYCHQHYST',
     'sig_res': 5,
     'start': (67, '68'),
     'end': (93, '94')}],
   'receptor': [{'seq': 'SSISSSNLHWYQQKSETSPKPWIYGTSNLASGVPVRFSGSGSGTSYSLTISSMEAEDAATYYCQQWN',
     'sig_res': 10,
     'start': (26, '27'),
     'end': (92, '93')}]},
  {'antibody': [{'seq': 'TGEPT',
     'sig_res': 3,
     'start': (54, '55'),
     'end': (58, '59')},
    {'seq': 'NYY', 'sig_res': 3, 'start': (103, '104'), 'end': (105, '106')}],
   'receptor': [{'seq': 'SSSNLHWYQQKSETSPKPWIYGTSN',
     'sig_res': 5,
     'start': (29, '30'),
     'end': (53, '54')},
    {'seq': 'NSYPY', 'sig_res': 4, 'start': (92, '93'), 'end': (96, '97')}]},
  {'antibody': [{'seq': 'TNYGMNWVKQAPGKGLKWMAWINTYTGEPT',
     'sig_res': 10,
     'start': (29, '30'),
     'end': (58, '59')},
    {'seq': 'YYENYY',
     'sig_res': 3,
     'start': (100, '101'),
     'end': (105, '106')}],
   'rec

## distance

In [133]:
# calculate distance
dist_file = p.build_dist()
print(dist_file)

./outputs/1I/1IAI/dist.p


In [134]:
from analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()

number of complex 4
antibody chain L ~ receptor chain M
antibody chain L ~ receptor chain I
antibody chain H ~ receptor chain M
antibody chain H ~ receptor chain I


In [135]:
# complex
p3.print_min_dist(0)

L-M
------------------------------
     res_no  res atom      value aa
0         1  ASP    C   7.385357  D
1         2  ILE    C   7.939900  I
2         3  VAL    C  11.120538  V
3         4  MET    C  12.654535  M
4         5  THR    C  16.760532  T
..      ...  ...  ...        ... ..
209     210  ASN    C  67.225297  N
210     211  ARG    C  68.432760  R
211     212  ASN    C  69.885633  N
212     213  GLU    C  69.606573  E
213     214  CYS    C  68.165889  C

[214 rows x 5 columns]
------------------------------
    res_no  res atom     value aa
26      27  SER    C  3.902428  S
27      28  SER    C  3.186657  S
28      29  ILE    C  2.870615  I
29      30  SER    C  3.098180  S
30      31  SER    C  3.563316  S
31      32  SER    C  4.761554  S
67      68  SER    C  4.518211  S
68      69  GLY    C  4.908164  G
92      93  ASN    C  4.870901  N


In [136]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs[0]

Sequence-Distance data: ./outputs/1I/1IAI/seq_dist.p.


[{'L': [{'seq': 'Y',
    'sig_res': 1,
    'start': (91, 92),
    'end': (91, 92),
    'cutoff': 3}],
  'M': [{'seq': 'I',
    'sig_res': 1,
    'start': (28, 29),
    'end': (28, 29),
    'cutoff': 3}]},
 {'L': [{'seq': 'D',
    'sig_res': 1,
    'start': (27, 28),
    'end': (27, 28),
    'cutoff': 4},
   {'seq': 'RTDFTFTINSVQAEDLAVYYCHQHYS',
    'sig_res': 3,
    'start': (67, 68),
    'end': (92, 93),
    'cutoff': 4}],
  'M': [{'seq': 'SSISS',
    'sig_res': 5,
    'start': (26, 27),
    'end': (30, 31),
    'cutoff': 4}]},
 {'L': [{'seq': 'QDVS',
    'sig_res': 3,
    'start': (26, 27),
    'end': (29, 30),
    'cutoff': 5},
   {'seq': 'RTDFTFTINSVQAEDLAVYYCHQHYST',
    'sig_res': 5,
    'start': (67, 68),
    'end': (93, 94),
    'cutoff': 5}],
  'M': [{'seq': 'SSISSS',
    'sig_res': 6,
    'start': (26, 27),
    'end': (31, 32),
    'cutoff': 5},
   {'seq': 'SGTSYSLTISSMEAEDAATYYCQQWN',
    'sig_res': 3,
    'start': (67, 68),
    'end': (92, 93),
    'cutoff': 5}]},
 {'L': [{